In [15]:
RSEED=42
CV=5

In [11]:
import numpy as np
import pandas as pd

df = pd.read_csv('./data/kickstarter_projects.csv')


In [130]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

numerical_fields = ['Goal', 'Pledged', 'Backers']

column_transformation = ColumnTransformer([
    ('scaling', StandardScaler(), numerical_fields)],
    remainder='passthrough'
)

X_transformed = pd.DataFrame(column_transformation.fit_transform(X), columns=numerical_columns + list(X.columns[len(numerical_columns):]))

y = df.State
X = df.drop('State', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, stratify=y, random_state=RSEED)

X_train.head()

,Goal,Pledged,Backers,Subcategory,Country,Launched,Deadline,Goal,Pledged,Backers
52741,-0.03095,0.009734,-0.024887,1041218511,The Creek When He Came Back,Film & Video,Narrative Film,United States,2012-05-03 20:51:14,2012-05-29
90349,-0.019731,-0.088031,-0.096182,1021334183,SWIM IN STYLE,Fashion,Fashion,United States,2013-04-04 17:12:30,2013-05-11
99525,-0.034402,-0.029479,-0.031468,1916745459,Making a Full-length Album with Meg Olsen!,Music,Indie Rock,United States,2013-06-12 22:39:20,2013-07-12
264523,0.025144,-0.09988,-0.117021,23143482,Night Out,Technology,Apps,United States,2015-12-16 19:18:30,2016-02-14
348400,-0.036841,0.068221,0.985294,850003102,Kickstarter Gold: The Secret Loves of Geek Gir...,Comics,Anthologies,Canada,2017-06-20 16:06:03,2017-07-11


In [145]:
%%time

import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


class KickAssClassifier:
    
    def fit(self, X, y):
        self._fit_best_random_forest(X, y)
        self._fit_best_knn(X, y)
    
    def _fit_best_random_forest(self, X, y):
        rn_param_grid = {
            'n_estimators': [50, 100, 150],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10, 20, 40, 80, 160],
            'min_samples_leaf': [1, 2, 4, 8, 16, 32, 64, 128],
            'max_features': ['auto', 'sqrt', 'log2']
        }
        self._grid_search(RandomForestClassifier(), rn_param_grid)
    
    def _fit_best_knn(self, X, y):
        knn_param_grid = {
            'n_neighbors': np.arange(1, 50, 2),
            "weights" : ["uniform", "distance"],
            "p" : np.arange(1, 3)
        }
        self._grid_search(KNeighborsClassifier(), knn_param_grid)
        
    def _grid_search(self, model, params):
        gs = GridSearchCV(model, params, cv=CV, scoring='accuracy', n_jobs=10)
        gs.fit(X[col_scale], y)
        print('Best score:', round(gs.best_score_, 3))
        print('Best parameters:', gs.best_params_)
        

model = KickAssClassifier()
model.fit(X_train, y_train)

KeyboardInterrupt: 

In [147]:
y.unique()

array(['Failed', 'Successful', 'Canceled', 'Suspended', 'Live'],
      dtype=object)